In [2]:
import pandas as pd
import numpy as np

In [4]:
year = "2011"

# Getting the List of Pulled Tweets
tweets_file_name = year+"alltweets.txt"
tweets_header_list = ['id','text']
tweets_df = pd.read_csv(tweets_file_name, names=tweets_header_list, sep='\t',error_bad_lines=False)

# Getting all the tweets we were supposed to have
data_header_list = ["id", "tweet_time","team","opponent", "week", "home_or_away","score","opponent_score","point_spread","over_under"]
data_df = pd.read_csv('nfl-tweets-v1.0/tweets.nfl.'+year+'.weekly.csv', names=data_header_list)

In [5]:
# Pulling out the list of tweet ids for each
all_ids = set(data_df["id"].tolist())
pulled_ids = set(tweets_df['id'].tolist())

In [8]:
# Finding what's missing
missing_tweets = all_ids-pulled_ids

In [10]:
print(len(missing_tweets), len(all_ids), len(pulled_ids))

179917 519240 339323


In [13]:
for idx, i in enumerate(a):
    print(i, idx)

1 0
2 1
3 2


# Looking Into GroupBy

In [23]:
d = {'week': [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4], 'team': ['a','a','b','b','a','a','b','b','a','a','b','b','a','a','b','b'],'score': [10,2,1,3,2,4,3,5,4,6,5,7,6,8,7,9]}
df = pd.DataFrame(data=d)
df.groupby(['week','team']).mean()

score
week team       
1    a         6
     b         2
2    a         3
     b         4
3    a         5
     b         6
4    a         7
     b         8

In [24]:
df.to_csv('test.csv',index=False)

In [25]:
import re
import nltk
#from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def get_sentiment(tweet):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(tweet)['compound']

def add_sentiment(df):
    df['sentiment'] = df['text'].apply(lambda x: get_sentiment(str(x)))
    return df

def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text

def make_twitter_df(year):
    year = str(year)
    tweets_file_name = year+"alltweets.txt"
    tweets_header_list = ['id','text']
    tweets_df = pd.read_csv(tweets_file_name, names=tweets_header_list, sep='\t',error_bad_lines=False)
    tweets_df = tweets_df.dropna()
    tweets_df['text'] = tweets_df['text'].apply(cleanTxt)
    tweets_df['id'] = tweets_df['id'].apply(int)
    
    data_header_list = ["id", "tweet_time","team","opponent", "week", "home_or_away","score","opponent_score","point_spread","over_under"]
    data_df = pd.read_csv('nfl-tweets-v1.0/tweets.nfl.'+year+'.weekly.csv', names=data_header_list)
    
    combined_df = pd.merge(data_df,tweets_df,on='id').dropna()
    sent_df = add_sentiment(combined_df)
    
    final_df = sent_df[["team", "week","sentiment","text"]]
    final_df = final_df.groupby(['team','week'],as_index=False)['sentiment'].mean()

    return final_df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/stanierc17/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [26]:
df = make_twitter_df(2012)

In [27]:
df.to_csv("2012sentiment.csv",index=False)